# Text Classification with TensorFlow, Keras, and Datalab


In this 5-minute quickstart tutorial, we use cleanlab to find potential label errors in a text classification dataset of [IMDB movie reviews](https://ai.stanford.edu/~amaas/data/sentiment/). This dataset contains 50,000 text reviews, each labeled with a binary sentiment polarity label indicating whether the review is positive (1) or negative (0). cleanlab will shortlist _hundreds_ of examples that confuse our ML model the most; many of which are potential label errors, edge cases, or otherwise ambiguous examples.

**Overview of what we'll do in this tutorial:**

- Build a simple TensorFlow & Keras neural network and wrap it with cleanlab's `KerasWrapperSequential`. This wrapper class makes *any* Keras/Tensorflow model compatible with scikit-learn and compute out-of-sample preddicted probabilites

- Use `Datalab` to identify various issues in the text dataset including label issues and near duplicates

<div class="alert alert-info">
Quickstart
<br/>
    
Already have (out-of-sample) `pred_probs` from a model trained on an existing set of labels? Maybe you have some numeric `features` as well? Run the code below to find any potential label errors in your dataset.

<div  class=markdown markdown="1" style="background:white;margin:16px">  
    
```ipython3 
from cleanlab import Datalab

lab = Datalab(data=your_dataset, label_name="column_name_of_labels")
lab.find_issues(pred_probs=your_pred_probs, features=your_features)

lab.report()
lab.get_issues()
```
    
</div>
</div>

## 1. Install required dependencies


You can use `pip` to install all packages required for this tutorial as follows:

```ipython3
!pip install sklearn tensorflow tensorflow-datasets
!pip install cleanlab[datalab]
# Make sure to install the version corresponding to this tutorial
# E.g. if viewing master branch documentation:
#     !pip install git+https://github.com/cleanlab/cleanlab.git
```

In [1]:
# Package installation (hidden on docs.cleanlab.ai).
# If running on Colab, may want to use GPU (select: Runtime > Change runtime type > Hardware accelerator > GPU)
# Package versions we used: tensorflow==2.9.1 scikit-learn==1.2.0 tensorflow_datasets==4.5.2

dependencies = ["cleanlab", "sklearn", "tensorflow", "tensorflow_datasets", "datasets"]

# Supress outputs that may appear if tensorflow happens to be improperly installed: 
import os 
import logging 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # suppress tensorflow log output 
logging.getLogger('tensorflow').setLevel(logging.FATAL) 

if "google.colab" in str(get_ipython()):  # Check if it's running in Google Colab
    %pip install cleanlab  # for colab
    cmd = ' '.join([dep for dep in dependencies if dep != "cleanlab"])
    %pip install $cmd
else:
    missing_dependencies = []
    for dependency in dependencies:
        try:
            __import__(dependency)
        except ImportError:
            missing_dependencies.append(dependency)

    if len(missing_dependencies) > 0:
        print("Missing required dependencies:")
        print(*missing_dependencies, sep=", ")
        print("\nPlease install them before running the rest of this notebook.")

In [2]:
import re 
import string 
import pandas as pd 
from sklearn.metrics import accuracy_score, log_loss 
from sklearn.model_selection import cross_val_predict 
import tensorflow as tf 
from tensorflow.keras import layers 
import tensorflow_datasets as tfds 

from cleanlab.models.keras import KerasWrapperSequential
from cleanlab.count import estimate_cv_predicted_probabilities
from cleanlab import Datalab

SEED = 123456  # for reproducibility 

In [3]:
# This cell is hidden from docs.cleanlab.ai 

import random 
import numpy as np 

pd.set_option("display.max_colwidth", None) 

tf.keras.utils.set_random_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

## 2. Load and preprocess the IMDb text dataset


This dataset is provided in TensorFlow's Datasets.


In [4]:
%%capture
raw_ds = tfds.load(name="imdb_reviews", split="train", batch_size=-1, as_supervised=True)

raw_texts, labels = tfds.as_numpy(raw_ds)

Metal device set to: Apple M2


In [5]:
num_classes = len(set(labels))
print(f"Classes: {set(labels)}")

Classes: {0, 1}


Let's print the first example in the train set.

In [6]:
i = 0
print(f"Example Label: {labels[i]}")
print(f"Example Text: {raw_texts[i]}")

Example Label: 0
Example Text: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."


The data is stored as two numpy arrays:

1. `raw_texts`for the movie reviews in text format,
2. `labels` for the labels.

<div class="alert alert-info">
Bringing Your Own Data (BYOD)?

You can easily replace the above with your own text dataset, and continue with the rest of the tutorial.

</div>

Next, we have to convert the text strings into vectors which are better suited as inputs for neural networks. 

The first step is to define a function to preprocess the text data by:

1. Converting it to lower case
2. Removing the HTML break tags: `<br />`
3. Removing any punctuation marks


In [7]:
def preprocess_text(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(stripped_html, f"[{re.escape(string.punctuation)}]", "")

Then, we use a `TextVectorization` layer to preprocess, tokenize, and vectorize our text data to a suitabable format for a neural network.

In [8]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=preprocess_text,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

Adapting `vectorize_layer` to the text data creates a mapping of each token (i.e. word) to an integer index. Subsequently, we can vectorize our text data in the train and test sets by using this mapping. 

In [9]:
vectorize_layer.reset_state()
vectorize_layer.adapt(raw_texts)

vectorized_texts = vectorize_layer(raw_texts).numpy()

Our subsequent neural network models will directly operate on elements of `vectorized_texts` in order to classify reviews.

## 3. Define a classification model and compute out-of-sample predicted probabilities

Here, we build a simple neural network for classification with TensorFlow and Keras. We will also wrap it with cleanlab's `KerasWrapperSequential` to make it compatible with sklearn. Note: you can wrap *any* existing Keras model this way, by just replacing `keras.Sequential` with `KerasWrapperSequential` in your code. 


In [10]:
model = KerasWrapperSequential(  
    [  
        tf.keras.Input(shape=(None,), dtype="int64"),
        layers.Embedding(max_features + 1, 16),
        layers.Dropout(0.2),
        layers.GlobalAveragePooling1D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes),
        layers.Softmax()
    ],  # outputs probability that text belongs to class 1
    compile_kwargs= {
      "optimizer":"adam",
      "loss":tf.keras.losses.SparseCategoricalCrossentropy(),
      "metrics":tf.keras.metrics.CategoricalAccuracy(),
    },
)

To find potential labeling errors, cleanlab requires a probabilistic prediction from your model for every datapoint. However, these predictions will be overfitted (and thus unreliable) for examples the model was previously trained on. cleanlab is intended to only be used with out-of-sample predicted probabilities, i.e., on examples held out from the model during the training.

Here, we show how to obtain the out-of-sample predicted probabilities from our Keras model defined above.

In [11]:
cv_n_folds = 3  # for efficiency; values like 5 or 10 will generally work better
num_epochs = 15  

In [12]:
pred_probs = estimate_cv_predicted_probabilities(
    vectorized_texts,
    labels,
    clf=model,
    cv_n_folds=cv_n_folds,
    clf_kwargs={"epochs": num_epochs},
)

Epoch 1/15
521/521 [==============================] - 6s 11ms/step - loss: 0.6605 - categorical_accuracy: 0.5296
Epoch 2/15
521/521 [==============================] - 5s 9ms/step - loss: 0.5220 - categorical_accuracy: 0.4845
Epoch 3/15
521/521 [==============================] - 4s 8ms/step - loss: 0.4102 - categorical_accuracy: 0.4870
Epoch 4/15
521/521 [==============================] - 5s 9ms/step - loss: 0.3454 - categorical_accuracy: 0.4917
Epoch 5/15
521/521 [==============================] - 5s 9ms/step - loss: 0.3046 - categorical_accuracy: 0.4943
Epoch 6/15
521/521 [==============================] - 4s 8ms/step - loss: 0.2742 - categorical_accuracy: 0.4948
Epoch 7/15
521/521 [==============================] - 4s 8ms/step - loss: 0.2508 - categorical_accuracy: 0.4953
Epoch 8/15
521/521 [==============================] - 4s 8ms/step - loss: 0.2308 - categorical_accuracy: 0.4947
Epoch 9/15
521/521 [==============================] - 4s 8ms/step - loss: 0.2136 - categorical_accuracy

## 4. Use datalab to find issues in your dataset

Based on the given labels, out-of-sample predicted probabilities and features, cleanlab can quickly help us identify poorly labeled instances in our text data.

Here, we use Datalab to find potential errors in our data. Datalab has several ways of loading the data. In this case, we’ll simply wrap the training features and noisy labels in a dictionary. 

In [13]:
data = {"texts": raw_texts, "labels": labels}

All that is need to audit your data is to call `find_issues()`. We can pass in the predicted probabilites obtained above, and also the `vectorized_texts` array as feature embeddings for our dataset so that Datalab can perform several types of issue checks on our data.

In [14]:
lab = Datalab(data, label_name="labels")
lab.find_issues(pred_probs=pred_probs, features=vectorized_texts)

Finding label issues ...
Finding outlier issues ...
Fitting OOD estimator based on provided features ...
Finding near_duplicate issues ...
Audit complete. 1741 issues found in the dataset.


After the issue checks are done, we can check out more information using the `report` method:

In [15]:
lab.report()

Here is a summary of the different kinds of issues found in the data:

    issue_type    score  num_issues
         label 0.940320        1492
near_duplicate 0.432059         249
       outlier 0.595218           0

(Note: A lower score indicates a more severe issue across all examples in the dataset.)


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.g. due to annotation error) are flagged as having label issues.
    

Number of examples with this issue: 1492
Overall dataset quality in terms of this issue: : 0.9403

Examples representing most severe instances of this issue:
       is_label_issue  label_score  given_label  predicted_label
22294            True     0.268942            1                0
5204             True     0.268942            1                0
15079            True     0.268960            1                0
21889            True     0.268974            1   

From the report, we observe that Datalab has identified many label issues in our dataset. We can view the label errors identified by Datalab and the label quality score for each example using the `get_issues` method.

In [16]:
label_issues = lab.get_issues("label")
label_issues.head()

,is_label_issue,label_score,given_label,predicted_label
0,False,0.730802,0,0
1,False,0.716673,0,0
2,True,0.284272,0,1
3,False,0.727785,1,1
4,False,0.530145,1,1


This method returns a dataframe containing a label quality score for each example. These numeric scores lie between 0 and 1, where lower scores indicate examples more likely to be mislabeled. The dataframe also contains a boolean column specifying whether or not each example is identified to have a label issue (indicating it is likely mislabeled).

We can get the subset of examples flagged with label issues, and also sort by label quality score to find the indices of the 10 most likely mislabeled examples in our dataset.

In [17]:
identified_label_issues = label_issues[label_issues["is_label_issue"] == True]
lowest_quality_labels = label_issues["label_score"].argsort()[:10].to_numpy()

In [18]:
print(
    f"cleanlab found {len(identified_label_issues)} potential label errors in the dataset.\n"
    f"Here are indices of the top 10 most likely errors: \n {lowest_quality_labels}"
)

cleanlab found 1492 potential label errors in the dataset.
Here are indices of the top 10 most likely errors: 
 [22294  5204 15079 21889 10676 11186 15174 10589 21492 18928]


Let's review some of the most likely label errors:


To help us inspect these datapoints, we define a method to print any example from the dataset. We then display some of the top-ranked label issues identified by `cleanlab`:


In [19]:
def print_as_df(index):
    return pd.DataFrame(
        {"texts": raw_texts[index], "labels": labels[index]},
        [index]
    )

Here's a review labeled as positive (1), but it should be negative (0).
Some noteworthy snippets extracted from the review text:

> - "...incredibly **awful** score..."
>
> - "...**worst** Foley work ever done."
>
> - "...script is **incomprehensible**..."
>
> - "...editing is just **bizarre**."
>
> - "...**atrocious** pan and scan..."
>
> - "...**incoherent mess**..."
>
> - "...**amateur** directing there."


In [20]:
print_as_df(22294)

,texts,labels
22294,"b'This movie is stuffed full of stock Horror movie goodies: chained lunatics, pre-meditated murder, a mad (vaguely lesbian) female scientist with an even madder father who wears a mask because of his horrible disfigurement, poisoning, spooky castles, werewolves (male and female), adultery, slain lovers, Tibetan mystics, the half-man/half-plant victim of some unnamed experiment, grave robbing, mind control, walled up bodies, a car crash on a lonely road, electrocution, knights in armour - the lot, all topped off with an incredibly awful score and some of the worst Foley work ever done.<br /><br />The script is incomprehensible (even by badly dubbed Spanish Horror movie standards) and some of the editing is just bizarre. In one scene where the lead female evil scientist goes to visit our heroine in her bedroom for one of the badly dubbed: ""That is fantastical. I do not understand. Explain to me again how this is..."" exposition scenes that litter this movie, there is a sudden hand held cutaway of the girl\'s thighs as she gets out of bed for no apparent reason at all other than to cover a cut in the bad scientist\'s ""Mwahaha! All your werewolfs belong mine!"" speech. Though why they went to the bother I don\'t know because there are plenty of other jarring jump cuts all over the place - even allowing for the atrocious pan and scan of the print I saw.<br /><br />The Director was, according to one interview with the star, drunk for most of the shoot and the film looks like it. It is an incoherent mess. It\'s made even more incoherent by the inclusion of werewolf rampage footage from a different film The Mark of the Wolf Man (made 4 years earlier, featuring the same actor but playing the part with more aggression and with a different shirt and make up - IS there a word in Spanish for ""Continuity""?) and more padding of another actor in the wolfman get-up ambling about in long shot.<br /><br />The music is incredibly bad varying almost at random from full orchestral creepy house music, to bosannova, to the longest piano and gong duet ever recorded. (Thinking about it, it might not have been a duet. It might have been a solo. The piano part was so simple it could have been picked out with one hand while the player whacked away at the gong with the other.) <br /><br />This is one of the most bewilderedly trance-state inducing bad movies of the year so far for me. Enjoy.<br /><br />Favourite line: ""Ilona! This madness and perversity will turn against you!"" How true.<br /><br />Favourite shot: The lover, discovering his girlfriend slain, dropping the candle in a cartoon-like demonstration of surprise. Rank amateur directing there.'",1


Here's a review labeled as positive (1), but it should be negative (0).
Some noteworthy snippets extracted from the review text:

> - "...film seems **cheap**."
>
> - "...unbelievably **bad**..."
>
> - "...cinematography is **badly** lit..."
>
> - "...everything looking **grainy** and **ugly**."
>
> - "...sound is so **terrible**..."


In [21]:
print_as_df(5204)

,texts,labels
5204,"b'This low-budget erotic thriller that has some good points, but a lot more bad one. The plot revolves around a female lawyer trying to clear her lover who is accused of murdering his wife. Being a soft-core film, that entails her going undercover at a strip club and having sex with possible suspects. As plots go for this type of genre, not to bad. The script is okay, and the story makes enough sense for someone up at 2 AM watching this not to notice too many plot holes. But everything else in the film seems cheap. The lead actors aren\'t that bad, but pretty much all the supporting ones are unbelievably bad (one girl seems like she is drunk and/or high). The cinematography is badly lit, with everything looking grainy and ugly. The sound is so terrible that you can barely hear what people are saying. The worst thing in this movie is the reason you\'re watching it-the sex. The reason people watch these things is for hot sex scenes featuring really hot girls in Red Shoe Diary situations. The sex scenes aren\'t hot they\'re sleazy, shot in that porno style where everything is just a master shot of two people going at it. The woman also look like they are refuges from a porn shoot. I\'m not trying to be rude or mean here, but they all have that breast implants and a burned out/weathered look. Even the title, ""Deviant Obsession"", sounds like a Hardcore flick. Not that I don\'t have anything against porn - in fact I love it. But I want my soft-core and my hard-core separate. What ever happened to actresses like Shannon Tweed, Jacqueline Lovell, Shannon Whirry and Kim Dawson? Women that could act and who would totally arouse you? And what happened to B erotic thrillers like Body Chemistry, Nighteyes and even Stripped to Kill. Sure, none of these where masterpieces, but at least they felt like movies. Plus, they were pushing the envelope, going beyond Hollywood\'s relatively prude stance on sex, sexual obsessions and perversions. Now they just make hard-core films without the hard-core sex.'",1


Here's a review labeled as positive (1), but it should be negative (0).
Some noteworthy snippets extracted from the review text:

> - "...hard to imagine a **boring** shark movie..."
>
> - "**Poor focus** in some scenes made the production seems **amateurish**."
>
> - "...**do nothing** to take advantage of..."
>
> - "...**far too few** scenes of any depth or variety."
>
> - "...just **look flat**...no contrast of depth..."
>
> - "...**introspective** and **dull**...constant **disappointment**."


In [22]:
print_as_df(15079)

,texts,labels
15079,"b'Like the gentle giants that make up the latter half of this film\'s title, Michael Oblowitz\'s latest production has grace, but it\'s also slow and ponderous. The producer\'s last outing, ""Mosquitoman-3D"" had the same problem. It\'s hard to imagine a boring shark movie, but they somehow managed it. The only draw for Hammerhead: Shark Frenzy was it\'s passable animatronix, which is always fun when dealing with wondrous worlds beneath the ocean\'s surface. But even that was only passable. Poor focus in some scenes made the production seems amateurish. With Dolphins and Whales, the technology is all but wasted. Cloudy scenes and too many close-ups of the film\'s giant subjects do nothing to take advantage of IMAX\'s stunning 3D capabilities. There are far too few scenes of any depth or variety. Close-ups of these awesome creatures just look flat and there is often only one creature in the cameras field, so there is no contrast of depth. Michael Oblowitz is trying to follow in his father\'s footsteps, but when you\'ve got Shark-Week on cable, his introspective and dull treatment of his subjects is a constant disappointment.'",1


We also obversed from the report above that Datalab has identified some near duplicates in our dataset. We can similarly get more information about it by calling the `get_issues` method and passing in `near_duplicate` as the argument. 

Then, we can sort the resulting DataFrame by the near duplicate score to see the most severe duplicate issues.

In [23]:
duplicate_issues = lab.get_issues("near_duplicate")
duplicate_issues.sort_values("near_duplicate_score").head()

,is_near_duplicate_issue,near_duplicate_score,near_duplicate_sets,distance_to_nearest_neighbor
21220,True,0.0,[22184],0.0
11892,True,0.0,[17362],0.0
16827,True,0.0,[16886],0.0
16842,True,0.0,"[24540, 1564]",0.0
16886,True,0.0,[16827],0.0


The results above show that the text at index 21220 is a near duplicate as the text at index 22184, let's print both of those to compare their similarity:

In [24]:
print_as_df([21220, 22184])

,texts,labels
21220,"b'Yes, he is! ...No, not because of Pintilie likes to undress his actors and show publicly their privies. Pintilie IS THE naked ""emperor"" - so to speak...<br /><br />It\'s big time for someone to state the truth. This impostor is a voyeur, a brat locked in an old man\'s body. His abundance of nude scenes have no artistic legitimacy whatsoever. It is 100% visual perversion: he gets his kicks by making the actors strip in the buff and look at their willies. And if he does this in front of the audience, he might eve get a hard-on! Did you know that, on the set of ""Niki Ardelean"", he used to embarrass poor Coca Bloss, by telling her: ""Oh, Coca, how I wanna f*** you!""? She is a great lady, very decent and sensitive, and she became unspeakably ashamed - to his petty satisfaction! And, as a worrying alarm signal about the degree of vulgarity and lack of education in Romanian audiences, so many people are still so foolish to declare these visual obscenities ""works of art""! Will anyone have ever the decency to expose the truth of it all?'",0
22184,"b'Yes, he is! ...No, not because of Pintilie likes to undress his actors and show publicly their privies. Pintilie IS THE naked ""emperor"" - so to speak...<br /><br />It\'s big time for someone to state the truth. This impostor is a voyeur, a brat locked in an old man\'s body. His abundance of nude scenes have no artistic legitimacy whatsoever. It is 100% visual perversion: he gets his kicks by making the actors strip in the buff and look at their willies. And if he does this in front of the audience, he might eve get a hard-on! Did you know that, on the set of ""Niki Ardelean"", he used to embarrass poor Coca Bloss, by telling her: ""Oh, Coca, how I wanna f*** you!""? She is a great lady, very decent and sensitive, and she became unspeakably ashamed - to his petty satisfaction! And, as a worrying alarm signal about the degree of vulgarity and lack of education in Romanian audiences, so many people are still so foolish to declare these visual obscenities ""works of art""! Will anyone have ever the decency to expose the truth of it all?'",0


We see that these two examples are indeed duplicates of one another! 

As demonstrated above, datalab can efficiently shortlist the most likely label errors to speed up your data cleaning process. With this list, you can decide whether to fix these label issues or remove ambiguous and duplicated examples from your dataset to obtain an improved, higher quality dataset for training your next ML model.


In [26]:
# Note: This cell is only for docs.cleanlab.ai, if running on local Jupyter or Colab, please ignore it.

label_issue_indices = [5204, 22294, 15079]  # check these examples were found in label issues
if not all(x in identified_label_issues.index for x in label_issue_indices):
    raise Exception("Some highlighted examples are missing from identified_label_issues.")

identified_duplicate_issues = duplicate_issues[duplicate_issues["is_near_duplicate_issue"] == True]
duplicate_issue_indices = [21220, 22184]  # check these examples were found in duplicates
if not all(x in identified_label_issues.index for x in label_issue_indices):
    raise Exception("Some highlighted examples are missing from identified_duplicate_issues.")